# Capturing Food Expenditure by Age and Income in the [Consumer Expenditure Survey](https://www.bls.gov/cex/)

Last week, we used the [cross-tabulations](https://www.bls.gov/cex/csxcross.htm) in the Consumer Expenditure Survey (CEX) to evaluate the market for senior restaurant patrons. This notebook captures a start-to-finish workflow for generating charts with this information. 

Unfortunately, the cross-tabs we want do not seem to be available via the API.  (The API offers spending by age of the reference person, and spending by income of the consumer unit. It does not offer the cross-tab of age by income.) Consequently, we will need to work with the Excel files that are prepackaged by BLS.  To do this, we need a new library called [`openpyxl`](https://openpyxl.readthedocs.io/en/stable/) that we have not already installed.  To do this, we can leverage the [`conda`](https://conda.io/docs/) package manager that was installed with [Miniconda](https://conda.io/miniconda.html). (Miniconda is just a minimalist installation of the [Anaconda](https://www.anaconda.com/) distribution, which comes with Python, development environments like Jupyter Lab, and a whole bunch of Python [packages](https://pypi.python.org/pypi) that are useful for scientific computing.)  To get `openpyxl`, go to the terminal, and submit the following command:

`sudo conda install openpyxl`

You will get prompted to confirm you want to install (which you should confirm), and that's it.  The next time you start up Python, the package will be available. The remainder of this example assumes you have installed it already.

In [1]:
import numpy as np               # base package for numerical analysis
import pandas as pd              # wraps and extends a lot of numpy operations with the handy DataFrame and Series objects
import bokeh.plotting as bp      # provides methods of plotting data
import bokeh.models as bm        # provides methods of manipulating bokeh plots
from IPython.display import HTML # allows us to render the HTML the underlies our bokeh plots
import openpyxl as xl            # allows us to work with excel workbooks

# Define locations for data and figures
data_dir = '../data/'
fig_dir = '../figs/'

# Define function to allow us to display charts in an iframe
def show_iframe(url, iheight=400, iwidth=1000):
    display_string = '<iframe src={url} width={w} height={h}></iframe>'.format(url=url, w=iwidth, h=iheight)
    print(display_string)
    return HTML(display_string)

## Download the Data

The first thing we need to do is acquire the data.  This is, of course, possible by simply clicking on the links. However, if you want to do this again, or remember exactly how you did it, code is a pretty unambiguous resource. Remember that, even though we are inside the Jupyter Lab environment, we still have shell access.  We can start up a shell in addition to this notebook, or we can use `'!'` inside the Notebook.  What does this mean? Well, let's check our current location in the file system.

In [2]:
!pwd

/home/choct155/projects/telling_stories_with_data/examples/cex/src


`pwd` is a shell command that we just used from inside the Notebook! The upshot is, *all* shell commands are available to us, including the ever useful [`wget`](https://www.gnu.org/software/wget/), a ridiculously useful package for retrieving assets from the internet. You might be asking yourself, just how do I use `wget`. It turns out that all of the documentation for shell commands is accessible right in the shell itself.  To access the [manual](https://www.gnu.org/software/bash/manual/), just use the `man`. In most cases, you simply need to submit the following:

`man [command of interest]`

We can use this functionality to explore `wget` (by executing `!man wget` in the cell below), but the print out is long (and there is some character repitition), so we will leave it commented out for now.

In [3]:
#!man wget

We do, however, want to use `wget` to capture our data, so let's create our list of URLs.  All of the files we want have a common directory location:  `https://www.bls.gov/cex/2016/CrossTabs/agebyinc/`.  Let's be efficient with our code, and use it as the base with which we can generate each file link. We also need the file names themselves, so we will put those in a list. (We can determine the file URLs by either hovering over the link, or looking at the underlying HTML.)

In [4]:
# Capture the base URL as a string
cex_base_url = 'https://www.bls.gov/cex/2016/CrossTabs/agebyinc/'

# Capture the file names as a list of strings
cex_files = ['xunder25.xlsx', 'x25to34.xlsx', 'x35to44.xlsx', 'x45to54.xlsx', 'x55to64.xlsx', 'x65orup.xlsx']

cex_base_url, cex_files

('https://www.bls.gov/cex/2016/CrossTabs/agebyinc/',
 ['xunder25.xlsx',
  'x25to34.xlsx',
  'x35to44.xlsx',
  'x45to54.xlsx',
  'x55to64.xlsx',
  'x65orup.xlsx'])

Now for something a little crazy. We are going to build our URLs with a [list comprehension](https://www.datacamp.com/community/tutorials/python-list-comprehension). Check the Python introduction for details, but the upshot is that we are going to use an implied loop to join `cex_base_url` to each of the files in `cex_files`.

In [5]:
cex_urls = [cex_base_url+f for f in cex_files]

cex_urls

['https://www.bls.gov/cex/2016/CrossTabs/agebyinc/xunder25.xlsx',
 'https://www.bls.gov/cex/2016/CrossTabs/agebyinc/x25to34.xlsx',
 'https://www.bls.gov/cex/2016/CrossTabs/agebyinc/x35to44.xlsx',
 'https://www.bls.gov/cex/2016/CrossTabs/agebyinc/x45to54.xlsx',
 'https://www.bls.gov/cex/2016/CrossTabs/agebyinc/x55to64.xlsx',
 'https://www.bls.gov/cex/2016/CrossTabs/agebyinc/x65orup.xlsx']

Now that we have our list, we can leverage one of the strongest features of [iPython](http://ipython.org/). We can do even better than just issuing shell commands from within the Notebook (or Qt console)(https://ipython.org/ipython-doc/3/interactive/qtconsole.html). We can combine python code with shell code. To demonstrate this functionality, consider the shell command, [`echo`](https://www.gnu.org/software/coreutils/manual/html_node/echo-invocation.html).  It will print back whatever you tell it to print.

In [6]:
!echo abc123

abc123


Again, this is being executed by a shell, and is just being passed back to the Notebook.  You can write the same thing in the terminal and get the same result. 

If we want to use a Python object, we just need to prepend the reference with `'$'`.

In [7]:
x = 'abc123'

!echo $x

abc123


Take a moment to let this sink in. We are now creating objects in the Python space, and then using them in the bash space. This is not a regular feature of the base Python language (at least not this directly). This interoperability is due to the enhancements and extensions that came with the IPython project.

We can do the same thing with our URL list.

In [8]:
for u in cex_urls:
    !echo $u

https://www.bls.gov/cex/2016/CrossTabs/agebyinc/xunder25.xlsx
https://www.bls.gov/cex/2016/CrossTabs/agebyinc/x25to34.xlsx
https://www.bls.gov/cex/2016/CrossTabs/agebyinc/x35to44.xlsx
https://www.bls.gov/cex/2016/CrossTabs/agebyinc/x45to54.xlsx
https://www.bls.gov/cex/2016/CrossTabs/agebyinc/x55to64.xlsx
https://www.bls.gov/cex/2016/CrossTabs/agebyinc/x65orup.xlsx


Since we can embed these references in bash commands, we can retrieve our data easily.  We want to make sure we deposit these files in our data folder, as opposed to the current folder, which should only hold source code. The commands we want to run will all have a simiar structure.  Here is an example for the first file:

`wget -O ../figs/xunder25.xlsx https://www.bls.gov/cex/2016/CrossTabs/agebyinc/xunder25.xlsx`

The `-O` flag is an option that let's us specify where we want to deposit the file, and what to call it. We will echo each command after it has been executed to make clear what we are asking the shell to do.

In [9]:
# Capture the workbook associated with each URL and deposit in the data directory
for i,u in enumerate(cex_urls):
    new_file = data_dir+cex_files[i]
    !wget -O $new_file $u
    !echo wget -O $new_file $u

--2018-02-04 19:35:08--  https://www.bls.gov/cex/2016/CrossTabs/agebyinc/xunder25.xlsx
Resolving www.bls.gov (www.bls.gov)... 23.36.83.144, 2600:1408:1d:383::18b9, 2600:1408:1d:396::18b9
Connecting to www.bls.gov (www.bls.gov)|23.36.83.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 15198 (15K) [application/vnd.openxmlformats-officedocument.spreadsheetml.sheet]
Saving to: ‘../data/xunder25.xlsx’

../data/xunder25.xl 100%[===================>]  14.84K  --.-KB/s    in 0.004s  

2018-02-04 19:35:08 (4.01 MB/s) - ‘../data/xunder25.xlsx’ saved [15198/15198]

wget -O ../data/xunder25.xlsx https://www.bls.gov/cex/2016/CrossTabs/agebyinc/xunder25.xlsx
--2018-02-04 19:35:09--  https://www.bls.gov/cex/2016/CrossTabs/agebyinc/x25to34.xlsx
Resolving www.bls.gov (www.bls.gov)... 23.36.83.144, 2600:1408:1d:383::18b9, 2600:1408:1d:396::18b9
Connecting to www.bls.gov (www.bls.gov)|23.36.83.144|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 17032

We can quickly confirm that our capture was successful, and that everything we need is in the data directory.

In [10]:
!ls -la ../data/

total 156
drwxr-xr-x 2 root root  4096 Feb  4 17:42 .
drwxr-xr-x 6 root root  4096 Feb  4 17:02 ..
-rw-r--r-- 1 root root    80 Feb  4 17:37 .~lock.x25to34.xlsx#
-rw-r--r-- 1 root root    80 Feb  4 17:42 .~lock.xunder25.xlsx#
-rw-r--r-- 1 root root 17032 Sep 14 14:03 x25to34.xlsx
-rw-r--r-- 1 root root 17111 Sep 14 14:03 x35to44.xlsx
-rw-r--r-- 1 root root 17132 Sep 14 14:03 x45to54.xlsx
-rw-r--r-- 1 root root 17062 Sep 14 14:03 x55to64.xlsx
-rw-r--r-- 1 root root 17019 Sep 14 14:03 x65orup.xlsx
-rw-r--r-- 1 root root 15198 Sep 14 14:02 xunder25.xlsx


## Capture Relevant Data from the Excel Workbooks

We are now ready to use `openpyxl`. Our goal is to extract the same two rows from each age-specific file.  The first row captures average annual food expenditures by income group, while the second captures the number of people in that segment (e.g. folks under 25 years of age that make between \$20,000 and \$29,999).  Inspection reveals that the expenditure and population data are available in each file in the same locations:

+ Expenditures:  `$C50:$K50`
+ Population:  `$C4:$K4`

The only exception is `xunder25.xlsx`. It has fewer income categories, and reaches only to columns `I`.  That last income group, in all cases, is an open-ended catch all. For `xunder25.xlsx` it is '\$40,000 and more', and for all other files it is '\$70,000 and more'. Before we extract, let's explore the `openpyxl` functionality a bit. We can load a workbook file and then interrogate the loaded workbook object. For example, what are the sheet names for `xunder25.xlsx`?

In [11]:
# Load workbook
under25 = xl.load_workbook('../data/xunder25.xlsx')

# Print sheet names
under25.sheetnames

['Table 3202']

Inspection reveals that this is indeed the sheet name.  More to the point, we can also read information from individual cells in a given worksheet.

In [12]:
# Capture value of cell A1
cell_a1 = under25['Table 3202']['A1']
print('A1 Value: {}'.format(cell_a1.value))

# Ask about the row, column and coordinates of cell A1
print('A1 Row: {}'.format(cell_a1.row))
print('A1 Column: {}'.format(cell_a1.column))
print('A1 Coordinates: {}'.format(cell_a1.coordinate))

A1 Value: Table 3202. Consumer units with reference person under age 25 by income before taxes: Average annual expenditures and characteristics, Consumer Expenditure Survey, 2015-2016
A1 Row: 1
A1 Column: A
A1 Coordinates: A1


We are looking to capture a row of information, and we can do this by looping over cells.  We can define this range of cells as a `cellObj`, and then loop over each cell within it. Here is a simple print out of the income categories in `xunder25.xlsx`.

In [13]:
# Define a convenient variable for our sheet
under25_sheet = under25['Table 3202']

# Define cellObj as a range
cellObj = under25_sheet['C3:I3']
print('Cell Object: {}'.format(cellObj))

# For each cell in cellObj (note that cellObj is a tuple and we are taking the first element of it)...
for c in cellObj[0]:
    # ... tell me the coordinate and the value
    print('\nCoordinate: {}; Value: {}'.format(c.coordinate, c.value))

Cell Object: ((<Cell 'Table 3202'.C3>, <Cell 'Table 3202'.D3>, <Cell 'Table 3202'.E3>, <Cell 'Table 3202'.F3>, <Cell 'Table 3202'.G3>, <Cell 'Table 3202'.H3>, <Cell 'Table 3202'.I3>),)

Coordinate: C3; Value: Less
than
$5,000

Coordinate: D3; Value: $5,000
to
$9,999

Coordinate: E3; Value: $10,000
to
$14,999

Coordinate: F3; Value: $15,000
to
$19,999

Coordinate: G3; Value: $20,000
to
$29,999

Coordinate: H3; Value: $30,000
to
$39,999

Coordinate: I3; Value: $40,000
and
more


The logic of how we can extract data should be coming into focus.  We can use specific cell references, and map the values of the cells to Python variables. Let's now loop across our files and create two pandas DataFrames. One will hold average annual food expenditure (`food`) and the other will hold population counts (`pop`).

In [14]:
# Create empty lists to hold data
food_list = []
pop_list = []

# For each file...
for f in cex_files:
    # ... create empty sublists to hold each row of food and pop data ...
    food_row = []
    pop_row = []
    # ... capture the path to that file ...
    tmp_path = data_dir + f
    # ... load the file ...
    tmp_wb = xl.load_workbook(tmp_path)
    # ... isolate the first sheet ...
    tmp_sheet_name = tmp_wb.sheetnames[0]
    tmp_sheet = tmp_wb[tmp_sheet_name]
    # ... define cell objects for the food and pop data ...
    # (note that we are selecting the first element of the tuple immediately)
    foodObj = tmp_sheet['C50:K50'][0]
    popObj = tmp_sheet['C4:K4'][0]
    # ... and for each element in the cell objects ...
    for i in range(len(foodObj)):
        # ... throw the value of each cell in the relevant row list...
        food_row.append(foodObj[i].value)
        pop_row.append(popObj[i].value)
    # ... then throw each row in the parent list
    food_list.append(food_row)
    pop_list.append(pop_row)
    
# Define column and row names
cols = ['<5000', '5000-9999', '10000-14999', '15000-19999', '20000-29999', '30000-39999', '40000-49999', '50000-69999', '>=70000']
rows = ['<25', '25-34', '35-44', '45-54', '55-64', '>65']

# Construct food and pop DataFrames
food = pd.DataFrame(food_list, columns = cols, index = rows)
pop = pd.DataFrame(pop_list, columns = cols, index = rows)

print('Average Annual Food Expenditures:\n', food)
print('\nPopulation Count:\n',pop)

Average Annual Food Expenditures:
         <5000  5000-9999  10000-14999  15000-19999  20000-29999  30000-39999  \
<25    2915.0     3511.0       3723.0       3857.0       4069.0       5342.0   
25-34  4719.0     4061.0       4407.0       4237.0       5167.0       5266.0   
35-44  4606.0     4648.0       4727.0       5011.0       5247.0       5776.0   
45-54  4045.0     4237.0       3370.0       4517.0       4852.0       5222.0   
55-64  4349.0     3582.0       3684.0       3626.0       4982.0       4956.0   
>65    3995.0     3504.0       3069.0       3273.0       4500.0       5629.0   

       40000-49999  50000-69999  >=70000  
<25         6548.0          NaN      NaN  
25-34       5091.0       6536.0   9232.0  
35-44       5945.0       6878.0  11862.0  
45-54       5877.0       6519.0  11812.0  
55-64       5399.0       6458.0  10301.0  
>65         5942.0       7077.0   9631.0  

Population Count:
         <5000  5000-9999  10000-14999  15000-19999  20000-29999  30000-39999  \
<25

## Plot Expenditures for Consumers with Incomes Greater than \$70,000

Now that we have our data in hand, we can start plotting the relationships we want to see. For our purposes, we are interested in three views related to consumers who have incomes in excess of \$70,000:

1. Average food expenditure by age
2. Population size by age
3. Total food expenditure by age (the product of average expenditure and population)

It's a bit easier to achieve these tasks if we isolate the series we want upfront, and calculate the data for view #3.

In [66]:
# Capture >70k consumer data in a new DataFrame
food70 = pd.DataFrame({
    'food': food['>=70000'],
    'pop': pop['>=70000']
})

# Note that we don't have data for folks under 25, so we can drop the NaN values
food70 = food70.dropna()

# Calculate total expenditure by age
# (Note that the population is in thousands, so we need to adjust. We want our final answer in billions)
food70['market'] = ((food70['pop'] * 1000) * food70['food']) / 1000000000

# Add an integer index (by pushing age into a normal variable) to ease plotting
food70 = food70.reset_index().rename(columns={'index':'age'})

# Add formatted values for easy reading
food70['food_lab'] = food70['food'].apply(lambda x: '${:,.0f}'.format(x))
food70['pop_lab'] = food70['pop'].apply(lambda x: '{:,}'.format(int(x)))
food70['market_lab'] = food70['market'].apply(lambda x: '${:,.1f}'.format(x))

food70

,age,food,pop,market,food_lab,pop_lab,market_lab
0,25-34,9232.0,7550.0,69.701600,"$9,232","7,550",$69.7
1,35-44,11862.0,10436.0,123.791832,"$11,862","10,436",$123.8
2,45-54,11812.0,12681.0,149.787972,"$11,812","12,681",$149.8
3,55-64,10301.0,9961.0,102.608261,"$10,301","9,961",$102.6
4,>65,9631.0,5379.0,51.805149,"$9,631","5,379",$51.8


At long last, we are in a position to generate reusable and interactive charts.  Let's execute one view at a time, making sure we provide the capability to manipulate the chart and get information when we hover over individual plot elements.

In [78]:
# Define location of output file
bp.output_file(fig_dir+'bokeh_food_age_income_spend.html')

# Capture data as a ColumnDataSource object (useful for labeling)
source = bm.ColumnDataSource(data=food70)

# Define plot tools
hover = bm.HoverTool(tooltips=[
    ("Age", "@age"),
    ("Avg Spend", "@food_lab")
])
plot_tools = "pan, reset"


# Create a figure upon which to draw our chart
fig = bp.figure(plot_width=600, plot_height=400, y_range=(0, 13000), tools=[hover, plot_tools])

# Plot data as vertical bars
# (Note that bokeh needs numeric values for the x axis, we can map age labels in after the fact)
fig.vbar(x='index', width=0.8, bottom=0, top='food', color='#6382b5', source=source)

# Annotate plot
fig.title.text = "Average Annual Expenditure on Food by Age"
fig.xaxis.axis_label = "Age"
labels = bm.LabelSet(x='index', y='food', text='food_lab', level='glyph', text_align='center',
                     y_offset=-30, render_mode='css', source=source, text_color='#ffffff', text_font_style='bold')
fig.add_layout(labels)

# Map x tick positions to labels (remember we dropped <25)
label_map = dict(zip(food70.index, food70['age']))
js_format = """
    var labels = {};
    return labels[tick];
""".format(label_map)
fig.xaxis.formatter = bm.FuncTickFormatter(code=js_format)

# Remove y axis
fig.yaxis.visible = False

# Remove grid lines
fig.xgrid.grid_line_color = None
fig.ygrid.grid_line_color = None
fig.outline_line_color = None

# Show the figure
bp.save(fig)

show_iframe(fig_dir+'bokeh_food_age_income_spend.html', iwidth=650, iheight=450)

<iframe src=../figs/bokeh_food_age_income_spend.html width=650 height=450></iframe>


In [83]:
# Define location of output file
bp.output_file(fig_dir+'bokeh_food_age_income_pop.html')

# Define plot tools
hover = bm.HoverTool(tooltips=[
    ("Age", "@age"),
    ("Avg Spend", "@pop_lab")
])
plot_tools = "pan, reset"


# Create a figure upon which to draw our chart
fig = bp.figure(plot_width=600, plot_height=400, y_range=(0, 13000), tools=[hover, plot_tools])

# Plot data as vertical bars
# (Note that bokeh needs numeric values for the x axis, we can map age labels in after the fact)
fig.vbar(x='index', width=0.8, bottom=0, top='pop', color='#890d13', source=source)

# Annotate plot
fig.title.text = "Population by Age"
fig.xaxis.axis_label = "Age"
labels = bm.LabelSet(x='index', y='pop', text='pop_lab', level='glyph', text_align='center',
                     y_offset=-30, render_mode='css', source=source, text_color='#ffffff', text_font_style='bold')
fig.add_layout(labels)

# Map x tick positions to labels (remember we dropped <25)
fig.xaxis.formatter = bm.FuncTickFormatter(code=js_format)

# Remove unnecessary lines
fig.yaxis.visible = False
fig.xgrid.grid_line_color = None
fig.ygrid.grid_line_color = None
fig.outline_line_color = None

# Show the figure
bp.save(fig)

show_iframe(fig_dir+'bokeh_food_age_income_pop.html', iwidth=650, iheight=450)

<iframe src=../figs/bokeh_food_age_income_pop.html width=650 height=450></iframe>


In [84]:
# Define location of output file
bp.output_file(fig_dir+'bokeh_food_age_income_market.html')

# Define plot tools
hover = bm.HoverTool(tooltips=[
    ("Age", "@age"),
    ("Avg Spend", "@market_lab")
])
plot_tools = "pan, reset"


# Create a figure upon which to draw our chart
fig = bp.figure(plot_width=600, plot_height=400, y_range=(0, 160), tools=[hover, plot_tools])

# Plot data as vertical bars
# (Note that bokeh needs numeric values for the x axis, we can map age labels in after the fact)
fig.vbar(x='index', width=0.8, bottom=0, top='market', color='#539e68', source=source)

# Annotate plot
fig.title.text = "Total Expenditure by Age"
fig.xaxis.axis_label = "Age"
labels = bm.LabelSet(x='index', y='market', text='market_lab', level='glyph', text_align='center',
                     y_offset=-30, render_mode='css', source=source, text_color='#ffffff', text_font_style='bold')
fig.add_layout(labels)

# Map x tick positions to labels (remember we dropped <25)
fig.xaxis.formatter = bm.FuncTickFormatter(code=js_format)

# Remove unnecessary lines
fig.yaxis.visible = False
fig.xgrid.grid_line_color = None
fig.ygrid.grid_line_color = None
fig.outline_line_color = None

# Show the figure
bp.save(fig)

show_iframe(fig_dir+'bokeh_food_age_income_market.html', iwidth=650, iheight=450)

<iframe src=../figs/bokeh_food_age_income_market.html width=650 height=450></iframe>
